In [1]:
# 1) Kütüphaneler
!pip install statsmodels scikit-learn matplotlib seaborn tqdm --quiet

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer
import statsmodels.api as sm
from google.colab import files

sns.set(context="notebook", style="whitegrid", rc={"figure.dpi":130})
plt.rcParams["figure.figsize"] = (9,6)
print("Kütüphaneler yüklendi ve içe aktarıldı.")

Kütüphaneler yüklendi ve içe aktarıldı.


In [2]:
# 2) Dosya yükleme (Colab) - çalıştırınca pencere açılır
print("CSV dosyalarını seç: WDICSV.csv, WDICountry.csv, WDISeries.csv, WDIcountry-series.csv, WDIfootnote.csv, WDIseries-time.csv")
uploaded = files.upload()

# 3) Dosyaları oku (isimler aynıysa)
wdi_main = pd.read_csv("WDICSV.csv", low_memory=False)
wdi_country = pd.read_csv("WDICountry.csv")
wdi_series = pd.read_csv("WDISeries.csv")
# opsiyonel/destek
try:
    wdi_country_series = pd.read_csv("WDIcountry-series.csv")
except:
    wdi_country_series = None
try:
    wdi_footnote = pd.read_csv("WDIfootnote.csv")
except:
    wdi_footnote = None
try:
    wdi_series_time = pd.read_csv("WDIseries-time.csv")
except:
    wdi_series_time = None

print("Dosyalar okundu:", wdi_main.shape, wdi_country.shape, wdi_series.shape)

CSV dosyalarını seç: WDICSV.csv, WDICountry.csv, WDISeries.csv, WDIcountry-series.csv, WDIfootnote.csv, WDIseries-time.csv


Saving WDICountry.csv to WDICountry.csv
Saving WDIcountry-series.csv to WDIcountry-series.csv
Saving WDICSV.csv to WDICSV.csv
Saving WDIfootnote.csv to WDIfootnote.csv
Saving WDISeries.csv to WDISeries.csv
Saving WDIseries-time.csv to WDIseries-time.csv
Dosyalar okundu: (403256, 69) (265, 31) (1516, 20)


In [3]:
# 4) Uzun formata çevir (WDICSV tipik format: yıllar sütun)
# Bulunan yıl sütunlarını tespit et
year_cols = [c for c in wdi_main.columns if c.strip().isdigit()]
id_vars = [c for c in wdi_main.columns if c not in year_cols]

wdi_long = wdi_main.melt(id_vars=id_vars, value_vars=year_cols,
                         var_name="Year", value_name="Value")

# temiz ve tip düzelt
wdi_long['Year'] = wdi_long['Year'].astype(int)
# Bazı WDI dosyalarında ".." gibi notlar olur:
wdi_long = wdi_long[wdi_long['Value'].notna()]
wdi_long = wdi_long[wdi_long['Value'] != ".."]
wdi_long['Value'] = pd.to_numeric(wdi_long['Value'], errors='coerce')
wdi_long.dropna(subset=['Value'], inplace=True)

# normalize column names if different (defensive)
for col in wdi_long.columns:
    # nothing
    pass

print("Veri uzun formata çevrildi ve ilk temizlik yapıldı.")

Veri uzun formata çevrildi ve ilk temizlik yapıldı.


In [9]:
import numpy as np
import pandas as pd

# ---------------------------------------------------------
# 5) Hedef göstergeleri seç
# ---------------------------------------------------------
wdi_long.rename(columns=lambda x: x.strip(), inplace=True)

GDP_CODE = 'NY.GDP.PCAP.CD'
CO2_PC_CODE = 'EN.GHG.CO2.PC.CE.AR5'  # kişi başı CO2
CO2_TOTAL_CODE = 'EN.ATM.CO2E.KT'     # toplam CO2
POP_CODE = 'SP.POP.TOTL'

target_codes = [GDP_CODE, CO2_PC_CODE, CO2_TOTAL_CODE, POP_CODE]
target = wdi_long[wdi_long['Indicator Code'].isin(target_codes)].copy()
print(f"Hedef göstergeler seçildi. Satır sayısı: {target.shape[0]}")

Hedef göstergeler seçildi. Satır sayısı: 45296


In [30]:
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 14.4 MB/s eta 0:00:00


In [31]:
# ==============================================================================
# Gerekli Kod Tanımları (Normalde Adım 5'te yapılır)
# Hata düzeltmesi için yeniden tanımlandı
# ==============================================================================
GDP_CODE = 'NY.GDP.PCAP.CD'
CO2_CODE = 'EN.ATM.CO2E.PC'
# Kullanıcının pivot çıktısında görülen alternatif kod:
CO2_FALLBACK_CODE = 'EN.GHG.CO2.PC.CE.AR5'


# ==============================================================================
# 6. PIVOT YAPMA, FİLTRELEME VE METADATA BİRLEŞTİRME (ADIM 6, 7, 8, 10)
# ==============================================================================

# 6) Pivot yap
pivot = target.pivot_table(index=['Country Name','Country Code','Year'],
                           columns='Indicator Code',
                           values='Value').reset_index()

# HATA DÜZELTMESİ: Sütun Adlarını Dinamik Olarak Eşleştir
rename_map = {}
if GDP_CODE in pivot.columns:
    rename_map[GDP_CODE] = 'gdp_per_capita'

# CO2 için öncelikle ana kodu, o yoksa fallback kodu kontrol et:
if CO2_CODE in pivot.columns:
    rename_map[CO2_CODE] = 'co2_per_capita'
elif CO2_FALLBACK_CODE in pivot.columns:
    rename_map[CO2_FALLBACK_CODE] = 'co2_per_capita'
else:
    # Bu durumda, her iki CO2 kodu da eksik demektir.
    # Adım 8'deki kontrolde zaten hata verecektir.
    pass

pivot = pivot.rename(columns=rename_map)
print(f"Pivot Tablo Sütunları (Adım 6 sonu): {pivot.columns.tolist()[-5:]}")


# 7) Filtre: Yıl aralığı 2000-2020
pivot = pivot[(pivot['Year']>=2000)&(pivot['Year']<=2020)]

# 8) %80 doluluk: ülke bazında (gdp & co2 ikilisine bak)
REQUIRED_COLS = ['gdp_per_capita', 'co2_per_capita']
check_cols = [c for c in REQUIRED_COLS if c in pivot.columns]

if len(check_cols) < 2:
    # Bu kontrol, dinamik eşleştirmeye rağmen CO2 sütununun oluşamadığı nadir durumu yakalar.
    raise ValueError(f"HATA: EKC analizi için kritik sütunlar eksik. Pivot tabloda sadece {check_cols} bulundu. Lütfen WDICSV dosyanızda {GDP_CODE} ve {CO2_CODE} verilerinin (2000-2020) olduğunu kontrol edin.")

valid_countries = []
grouped = pivot.groupby('Country Name')
for country, g in grouped:
    # Şimdi sadece var olan sütunları (ikisi de var olmalı) kontrol et:
    both_nonnull = g[check_cols].notna().all(axis=1).mean()
    if both_nonnull >= 0.8:
        valid_countries.append(country)

pivot = pivot[pivot['Country Name'].isin(valid_countries)].copy()
print(f"Ülke sayısı (>=%80 doluluk): {pivot['Country Name'].nunique()}")

# 9) Ek temizlik ve dönüşümler
pivot = pivot[pivot['gdp_per_capita']>0] # GSYİH <=0 satırlarını çıkar
low_q = pivot['co2_per_capita'].quantile(0.01)
high_q = pivot['co2_per_capita'].quantile(0.99)
pivot['co2_wins'] = pivot['co2_per_capita'].clip(lower=low_q, upper=high_q) # Winsorize
pivot['log_gdp_per_capita'] = np.log(pivot['gdp_per_capita']) # Log dönüşümü
pivot['log_gdp_sq'] = pivot['log_gdp_per_capita']**2 # Kare terim

# 10) Country metadata (bölge bilgisi) ekleme
key_country = None
if 'Country Code' in wdi_country.columns:
    key_country = 'Country Code'
elif 'CountryCode' in wdi_country.columns:
    key_country = 'CountryCode'

if key_country:
    pivot = pivot.merge(wdi_country, left_on='Country Code', right_on=key_country, how='left')
    print("Ülke meta verileri eklendi.")

# ==============================================================================
# 9. & 11. MODELLEME VE PANEL ANALİZİ
# ==============================================================================

# Prepare data for modeling (drop NA)
model_df = pivot[['Country Name','Year','log_gdp_per_capita','log_gdp_sq','co2_wins']].dropna().copy()


# 9.1 A) OLS (pooled cross-section/time) - EKC Temel Testi
X_ols = sm.add_constant(model_df[['log_gdp_per_capita','log_gdp_sq']])
y = model_df['co2_wins']
ols_model = sm.OLS(y, X_ols).fit(cov_type='HC3')  # robust SEs

print("\n--- 9.1 OLS (pooled) summary (EKC Temel Form) ---")
print(ols_model.summary().as_text())


# 9.2 B) Polynomial + Ridge with CV (Hata Metriği İçin Çalıştırılır)
X = model_df[['log_gdp_per_capita']].values
y = model_df['co2_wins'].values

def rmse_cv(estimator, X, y):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    neg_mse = cross_val_score(estimator, X, y, scoring='neg_mean_squared_error', cv=kf)
    return np.sqrt(-neg_mse.mean())

degrees = [1,2,3]
alphas = [0.0, 0.1, 1.0, 10.0, 100.0]
best = {'score':np.inf}
for deg in degrees:
    for a in alphas:
        steps = []
        steps.append(('poly', PolynomialFeatures(degree=deg, include_bias=False)))
        steps.append(('ridge', Ridge(alpha=a)))
        pipe = Pipeline(steps)
        score = rmse_cv(pipe, X, y)
        if score < best['score']:
            best = {'degree':deg, 'alpha':a, 'score':score, 'pipe':pipe}

print("\n--- 9.2 Best model (sklearn pipeline) ---")
print(f"Best model (Ridge/CV): deg={best['degree']} alpha={best['alpha']} RMSE={best['score']:.4f}")
best_pipe = best['pipe']
best_pipe.fit(X,y)
model_df['pred_ridge'] = best_pipe.predict(model_df[['log_gdp_per_capita']])
model_df['residual'] = model_df['co2_wins'] - model_df['pred_ridge']


# Gerekli kütüphaneleri bu bloğun başında tekrar içe aktar (Hata çözümü)
import numpy as np
import linearmodels.panel
from linearmodels.panel import PanelOLS, RandomEffects
from scipy.stats import chi2

# 11.1) Panel veri hazırlık
# Burada 'pivot' DataFrame'inin önceki adımlardan geldiği varsayılır.
panel = pivot.set_index(['Country Name','Year'])
panel_data = panel[['co2_wins','log_gdp_per_capita','log_gdp_sq']].dropna()

# 11.2) Panel modeli: FE vs RE karşılaştırması
# Hata düzeltmesi sonrası, doğrudan sınıf adlarını kullanıyoruz.
fe_model = PanelOLS.from_formula('co2_wins ~ log_gdp_per_capita + log_gdp_sq + EntityEffects', data=panel_data).fit(cov_type='robust')
re_model = RandomEffects.from_formula('co2_wins ~ 1 + log_gdp_per_capita + log_gdp_sq', data=panel_data).fit(cov_type='robust')

print("\n--- 11.2 Fixed Effects Summary (Sabit Etkiler) ---")
print(fe_model.summary)
print("\n--- 11.2 Random Effects Summary (Rastgele Etkiler) ---")
print(re_model.summary)


# 11.3) Hausman Testi (FE vs RE seçimi)
def hausman(fe, re):
    b = fe.params.values
    B = re.params.values[1:]
    Vb = fe.cov.values
    VB = re.cov.values[1:, 1:]

    diff = b - B
    try:
        stat = diff.T @ np.linalg.inv(Vb - VB) @ diff
    except np.linalg.LinAlgError:
        print("Uyarı: Kovaryans matrisi farkı ters çevrilemedi.")
        return np.nan, np.nan

    df = len(b)
    pval = 1 - chi2.cdf(stat, df)
    return stat, pval

stat, pval = hausman(fe_model, re_model)
print(f"\n--- 11.3 Hausman Test Sonucu ---")
print(f"Hausman Test: stat={stat:.2f}, p-value={pval:.4f}")
if pval < 0.05:
    print("Hausman Kararı: FE tercih edilir (p<0.05).")
    final_model = fe_model
else:
    print("Hausman Kararı: RE tercih edilir (p>=0.05).")
    final_model = re_model

print(f"\n✅ RAPOR İÇİN NİHAİ MODEL TİPİ: {final_model.model.__class__.__name__}")

Pivot Tablo Sütunları (Adım 6 sonu): ['Country Code', 'Year', 'co2_per_capita', 'gdp_per_capita', 'SP.POP.TOTL']
Ülke sayısı (>=%80 doluluk): 245
Ülke meta verileri eklendi.

--- 9.1 OLS (pooled) summary (EKC Temel Form) ---
                            OLS Regression Results                            
Dep. Variable:               co2_wins   R-squared:                       0.413
Model:                            OLS   Adj. R-squared:                  0.412
Method:                 Least Squares   F-statistic:                     1665.
Date:                Fri, 31 Oct 2025   Prob (F-statistic):               0.00
Time:                        19:48:32   Log-Likelihood:                -14766.
No. Observations:                5136   AIC:                         2.954e+04
Df Residuals:                    5133   BIC:                         2.956e+04
Df Model:                           2                                         
Covariance Type:                  HC3                           

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PolynomialFeatures was fitted without feature names
  warnings.warn(



--- 11.2 Fixed Effects Summary (Sabit Etkiler) ---
                          PanelOLS Estimation Summary                           
Dep. Variable:               co2_wins   R-squared:                        0.0439
Estimator:                   PanelOLS   R-squared (Between):              0.4670
No. Observations:                5136   R-squared (Within):               0.0439
Date:                Fri, Oct 31 2025   R-squared (Overall):              0.4612
Time:                        19:48:33   Log-likelihood                   -6661.3
Cov. Estimator:                Robust                                           
                                        F-statistic:                      112.11
Entities:                         245   P-value                           0.0000
Avg Obs:                       20.963   Distribution:                  F(2,4889)
Min Obs:                       19.000                                           
Max Obs:                       21.000   F-statistic (robu

In [32]:
# 12) Model interpret & country-level residuals
model_df['pred_ridge'] = best_pipe.predict(model_df[['log_gdp_per_capita']])
model_df['residual'] = model_df['co2_wins'] - model_df['pred_ridge']

# Aggregate for countries: check trend 2000->2020 for green growth candidate detection
agg2000 = pivot[pivot['Year']==2000].set_index('Country Name')[['gdp_per_capita','co2_per_capita']]
agg2020 = pivot[pivot['Year']==2020].set_index('Country Name')[['gdp_per_capita','co2_per_capita']]
agg = pd.DataFrame(index=agg2000.index)
agg['gdp2000'] = agg2000['gdp_per_capita']
agg['gdp2020'] = agg2020['gdp_per_capita']
agg['co22000'] = agg2000['co2_per_capita']
agg['co22020'] = agg2020['co2_per_capita']
agg = agg.dropna()
agg['gdp_change'] = agg['gdp2020'] - agg['gdp2000']
agg['co2_change'] = agg['co22020'] - agg['co22000']
green_candidates = agg[(agg['gdp_change']>0)&(agg['co2_change']<0)].copy()
green_candidates = green_candidates.sort_values('gdp_change', ascending=False)
print("\nYeşil büyüme adayları (örnek):", green_candidates.head())


Yeşil büyüme adayları (örnek):                    gdp2000        gdp2020    co22000    co22020    gdp_change  \
Country Name                                                                    
Luxembourg    48659.598875  116860.028172  20.171212  12.604157  68200.429297   
Ireland       26334.567205   86622.506725  11.516609   6.647496  60287.939520   
Switzerland   38865.021940   85897.784334   6.235849   4.154446  47032.762394   
Singapore     23852.838951   61410.079263  11.301037   9.476104  37557.240312   
Denmark       30722.025184   60985.488560   9.954986   4.766725  30263.463376   

              co2_change  
Country Name              
Luxembourg     -7.567056  
Ireland        -4.869112  
Switzerland    -2.081403  
Singapore      -1.824933  
Denmark        -5.188260  


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PolynomialFeatures was fitted without feature names
  warnings.warn(


In [33]:
# 13) Görseller: 3 etkili görsel
os.makedirs("outputs", exist_ok=True)

# 13.1 Global scatter + fitted curve (using dense X)
xx = np.linspace(model_df['log_gdp_per_capita'].min(), model_df['log_gdp_per_capita'].max(), 200).reshape(-1,1)
yy = best_pipe.predict(xx)

plt.figure(figsize=(9,6))
sns.scatterplot(x='log_gdp_per_capita', y='co2_wins', data=model_df, alpha=0.25)
plt.plot(xx, yy, linewidth=2, label=f'Poly Ridge deg={best["degree"]} alpha={best["alpha"]}')
plt.xlabel('log(GDP per capita)')
plt.ylabel('CO2 per capita (winsorized)')
plt.title('EKC - Pooled data 2000-2020 (scatter + fitted)')
plt.legend()
plt.tight_layout()
plt.savefig("outputs/ekc_scatter_fitted.png")
plt.close()

# 13.2 Time-series for top 6 green candidates (CO2 trend vs GDP)
top6 = list(green_candidates.head(6).index)
ts = pivot[pivot['Country Name'].isin(top6)].copy()
# plot CO2 and GDP in subplots
for country in top6:
    sub = ts[ts['Country Name']==country]
    fig, ax1 = plt.subplots(figsize=(8,4))
    ax1.plot(sub['Year'], sub['co2_per_capita'], marker='o', label='CO2 per cap', linestyle='-')
    ax1.set_ylabel('CO2 per capita')
    ax2 = ax1.twinx()
    ax2.plot(sub['Year'], sub['gdp_per_capita'], marker='x', color='tab:orange', label='GDP per cap')
    ax2.set_ylabel('GDP per capita (USD)')
    plt.title(f"{country} - CO2 and GDP (2000-2020)")
    fig.tight_layout()
    file = f"outputs/ts_{country.replace(' ','_')}.png"
    plt.savefig(file)
    plt.close()

# 13.3 Regional heatmap: avg CO2 by region (using wdi_country region column heuristically)
# attempt to find region column (common names: 'Region', 'Region Name', 'RegionName','RegionCode')
region_col = None
for candidate in ['Region','Region Name','RegionName','RegionCode','Region code','region']:
    if candidate in pivot.columns:
        region_col = candidate
        break

if region_col:
    reg = pivot.groupby([region_col,'Year'])['co2_per_capita'].mean().reset_index()
    # pivot to heatmap: rows region, cols year
    hp = reg.pivot(index=region_col, columns='Year', values='co2_per_capita').fillna(0)
    plt.figure(figsize=(12,6))
    sns.heatmap(hp, cmap='viridis', linewidths=.5)
    plt.title('Avg CO2 per capita by Region (2000-2020)')
    plt.tight_layout()
    plt.savefig("outputs/regional_co2_heatmap.png")
    plt.close()
    print("3 temel görsel (scatter, zaman serileri, heatmap) outputs klasörüne kaydedildi.")
else:
    print("Bölge (Region) sütunu bulunamadı; ısı haritası atlandı.")

3 temel görsel (scatter, zaman serileri, heatmap) outputs klasörüne kaydedildi.


In [34]:
# 14) Kaydet: temizlenmiş veri, model sonuçları, green candidates
pivot.to_csv("outputs/WDI_EKC_Cleaned_2000_2020.csv", index=False)
model_df.to_csv("outputs/WDI_EKC_ModelData.csv", index=False)
green_candidates.to_csv("outputs/Green_Growth_Candidates_2000_2020.csv")

# For Tableau & Power BI: save both "row-level cleaned" and aggregated summaries
# Tableau prefers denormalized row-level; PowerBI is fine with same CSV
pivot.to_csv("outputs/Tableau_WDI_EKC.csv", index=False)
pivot.to_csv("outputs/PowerBI_WDI_EKC.csv", index=False)

# Zip outputs for easy download
import zipfile
zf_name = "WDI_EKC_outputs.zip"
with zipfile.ZipFile(zf_name, 'w') as zf:
    for root, _, files_ in os.walk('outputs'):
        for f in files_:
            zf.write(os.path.join(root,f), arcname=f)
    # also include key CSVs
    zf.write("outputs/WDI_EKC_Cleaned_2000_2020.csv", arcname="WDI_EKC_Cleaned_2000_2020.csv")
    zf.write("outputs/WDI_EKC_ModelData.csv", arcname="WDI_EKC_ModelData.csv")
    zf.write("outputs/Green_Growth_Candidates_2000_2020.csv", arcname="Green_Growth_Candidates_2000_2020.csv")

print("\n✅ Pipeline tamamlandı. Üretilen dosyalar 'outputs/' klasöründe ve ZIP: ", zf_name)
files.download(zf_name)

# 15) Kısa özet çıktı (ekran)
print("\nÖzet:")
print("- Cleaned row-level CSV: outputs/WDI_EKC_Cleaned_2000_2020.csv")
print("- Model data (with predictions/residuals): outputs/WDI_EKC_ModelData.csv")
print("- Green growth candidates: outputs/Green_Growth_Candidates_2000_2020.csv")
print("- Görseller: outputs/ekc_scatter_fitted.png, outputs/regional_co2_heatmap.png, outputs/ts_<country>.png")


✅ Pipeline tamamlandı. Üretilen dosyalar 'outputs/' klasöründe ve ZIP:  WDI_EKC_outputs.zip


/usr/lib/python3.12/zipfile/__init__.py:1624: UserWarning: Duplicate name: 'WDI_EKC_Cleaned_2000_2020.csv'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/lib/python3.12/zipfile/__init__.py:1624: UserWarning: Duplicate name: 'WDI_EKC_ModelData.csv'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/lib/python3.12/zipfile/__init__.py:1624: UserWarning: Duplicate name: 'Green_Growth_Candidates_2000_2020.csv'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Özet:
- Cleaned row-level CSV: outputs/WDI_EKC_Cleaned_2000_2020.csv
- Model data (with predictions/residuals): outputs/WDI_EKC_ModelData.csv
- Green growth candidates: outputs/Green_Growth_Candidates_2000_2020.csv
- Görseller: outputs/ekc_scatter_fitted.png, outputs/regional_co2_heatmap.png, outputs/ts_<country>.png
